<a href="https://colab.research.google.com/github/omkart0124/Exploratory-Data-Analysis/blob/main/Stroke_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("stroke-data.csv")
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [ ]:
df.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [ ]:
df.drop(columns='id', axis=1, inplace=True)

As we can see in the above list of datatypes, columns named gender, ever_married, work_type, Residence_type, smoking_status all are of object data type which need to be converted to numerical values before providing them for training the model. So here we mapped them to numerical values such as 1 or 0.

In [ ]:
#df["Residence_type"] = df["Residence_type"].apply(lambda x: 1 if x == "Urban" else 0)
#df["ever_married"] = df["Residence_type"].apply(lambda x: 1 if x == "Yes" else 0)
#df["gender"] = df["gender"].apply(lambda x: 1 if x == "Male" else 0)
#df = pd.get_dummies(data=df, columns=['smoking_status', 'work_type'])

In [ ]:
#df.head(2)

In [ ]:
gender = pd.get_dummies(df["gender"], drop_first=True)
ever_married = pd.get_dummies(df["ever_married"], drop_first=True)
Residence_type = pd.get_dummies(df["Residence_type"], drop_first=True)
smoking_status = pd.get_dummies(df["smoking_status"], drop_first=True)
work_type = pd.get_dummies(df["work_type"], drop_first=True)

In [ ]:
# delete extra features
df.drop(["gender", "ever_married","Residence_type","smoking_status","work_type"], axis=1, inplace=True)

In [ ]:
df1 = pd.concat([df, gender, ever_married, Residence_type ,smoking_status ,work_type,], axis=1)

In [ ]:
df1.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,Male,Other,Yes,Urban,formerly smoked,never smoked,smokes,Never_worked,Private,Self-employed,children
0,67.0,0,1,228.69,36.6,1,1,0,1,1,1,0,0,0,1,0,0
1,61.0,0,0,202.21,NaN,1,0,0,1,0,0,1,0,0,0,1,0
2,80.0,0,1,105.92,32.5,1,1,0,1,0,0,1,0,0,1,0,0
3,49.0,0,0,171.23,34.4,1,0,0,1,1,0,0,1,0,1,0,0
4,79.0,1,0,174.12,24.0,1,0,0,1,0,0,1,0,0,0,1,0


In [ ]:
df1.isnull().sum()

age                    0
hypertension           0
heart_disease          0
avg_glucose_level      0
bmi                  201
stroke                 0
Male                   0
Other                  0
Yes                    0
Urban                  0
formerly smoked        0
never smoked           0
smokes                 0
Never_worked           0
Private                0
Self-employed          0
children               0
dtype: int64

we need to ensure that our dataset does not contain any null values. But as we can see above "bmi"column has null values which we have filled using mean of the column.

In [ ]:
df1['bmi'] = df1['bmi'].fillna(df1['bmi'].mean())

In [ ]:
df1.isnull().sum()

age                  0
hypertension         0
heart_disease        0
avg_glucose_level    0
bmi                  0
stroke               0
Male                 0
Other                0
Yes                  0
Urban                0
formerly smoked      0
never smoked         0
smokes               0
Never_worked         0
Private              0
Self-employed        0
children             0
dtype: int64

##Train_Test_Split

In [ ]:
x = df1.drop('stroke', axis='columns')
y= df1['stroke']

In [ ]:
# scalling

ss = StandardScaler()
x = ss.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
x_train.shape

(3577, 16)

In [ ]:
model = tf.keras.Sequential(
    [tf.keras.layers.Dense(16, activation="relu", input_shape=(x.shape[1],)),#---------------1st layer the number 15 denotes number of neurons
     tf.keras.layers.Dense(25, activation="relu"),#---------------2st layer 20 neurons
     tf.keras.layers.Dense(1, activation="sigmoid")]#--------------Output layer 1 neuron
)

In [ ]:
model.summary ()

# params : input parameter along with bias.
# 16(No of features)+15(Neurons)+15(Bias)=255

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_16 (Dense)             (None, 25)                425       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 26        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer= 'adam', loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
trained_model = model.fit(x_train, y_train, epochs=100, batch_size=20)

Epoch 1/100
179/179 [==============================] - 1s 1ms/step - loss: 0.5404 - accuracy: 0.7107
Epoch 2/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1849 - accuracy: 0.9556
Epoch 3/100
179/179 [==============================] - 0s 983us/step - loss: 0.1965 - accuracy: 0.9466
Epoch 4/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1717 - accuracy: 0.9516
Epoch 5/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1577 - accuracy: 0.9542
Epoch 6/100
179/179 [==============================] - 0s 965us/step - loss: 0.1658 - accuracy: 0.9490
Epoch 7/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1497 - accuracy: 0.9550
Epoch 8/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1584 - accuracy: 0.9521
Epoch 9/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1743 - accuracy: 0.9447
Epoch 10/100
179/179 [==============================] - 0s 1ms/step - loss: 0.1436 - ac

In [ ]:
y_hat = model.predict(x_test)
y_hat

array([[0.00031951],
       [0.00065061],
       [0.03360289],
       ...,
       [0.00016311],
       [0.03863752],
       [0.03170586]], dtype=float32)

In [ ]:
y_hat1 = np.where(y_hat >= 0.5, 1, 0)
y_hat1.flatten()

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
roc_auc_score(y_test, y_hat1)

0.5347146323842692

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score)

In [ ]:
confusion_matrix(y_test,y_hat1)

array([[1442,   17],
       [  68,    6]])

In [ ]:
print(accuracy_score(y_test, y_hat1))

0.9445531637312459
